In [1]:
from Rings import Zomega, Exact_U, Ztau
from Circuits import FTCircuit
import numpy as np
import cmath

In [2]:
omega = Zomega(0,1,0,0)
tau = Zomega(0,0,1,-1)

T = Exact_U(Zomega(1,0,0,0), Zomega(0,0,0,0), 6)
F = Exact_U(tau, Zomega(1,0,0,0), 0)


In [3]:
def exact_synthesize(U):
    g = U.mu()
    V = U
    Circuit = FTCircuit()
    while g.value() >= 2: 
        
        min = 0
        min_value = V.j_mu(min).value()
        for j in range(0,10):
            if V.j_mu(j).value() < min_value:
                min = j
                min_value = V.j_mu(min).value()

        V = F*(T**min)*V
        g = V.mu()
        print("g value = " + str(g.value()))
        if min != 0:
            Circuit.join("T", 10 - min)
        Circuit.join("F", 1)
    
    # first factor out the phase of omega^k
    for k in range(10):
        if omega**k == V.u:
            Circuit.addPhase(omega**k)
            V = Exact_U(omega**(10-k), Zomega(0,0,0,0), 5 - 2*k) * V
            break
    
    # Am left with a matrix of the form U(1, 0, k) which we can match to T^j
    if V.k%10 != 5:
        Circuit.join("T",(V.k)%10)
    
    return Circuit

In [4]:
# u,v for compiling to precision 10^(-4)
u = Zomega(-255009288268180616, 341634187605978540, -140162031400595300, -70979508258857508)
v = Zomega(45639281038229241, -245501868762956335, 323384460018115951, -171655960820992834)
print("u,v value: " + str(u.value()) + ", " + str(v.value()))
print(u.N())

u,v value: (0.2225209323536271693057910339501718960403235379192 - 0.97492790351521088617128993084210791130398834268041j), (-0.000095335830488718436690483900585956421013102931046227 + 0.00013931168433793618073673226080873866489692634533319j)
294405716540324035696780176464778496


In [5]:
# Testing exact_synthesize for large u,v
large_exact = Exact_U(u,v,0)
synth = exact_synthesize(large_exact)
print(synth)
print(synth.numerical_approximate())

# each step decrementing g takes about 0.4 secs right now
# approximating the value of tau doesn't help

g value = 36909390162156359179612589055220586.617909117424803
g value = 7375464969799696706540889831675712.3262889458727080
g value = 1125306324272392223289759670640532.3392678926660555
g value = 320324146002500882359290733618561.60682377344875529
g value = 50812497408350794329162281658412.851795713691575298
g value = 10774718367722231745764925381162.401065747202664901
g value = 2150702345964948160531426569231.5863451937845394598
g value = 326628258412698204893085851542.69354817406034284379
g value = 96402665197506044846857964752.309503470961275081438
g value = 13798237934685888286946181702.175581730832573942025
g value = 2908067505872196716778858362.2509207382997564353652
g value = 564158522489636731121402486.35163886115575758602942
g value = 76755746355744697663960632.089558184064286065704500
g value = 10318167229823397219903076.397670927487554727502224
g value = 1290319613789353251856982.2061677567920722616026217
g value = 270593971685436435768616.17997371644604514631886020
g value 

In [4]:
'''
possible_G_values = []
for i in range(10):
    for j in range(10):
        for k in range(10):
            m = Exact_U(Zomega(0,1,0,0)**j*Zomega.fromTau(Ztau(0,1)), Zomega(0,1,0,0)**i, k)
            m = m * F
            if m.G() not in possible_G_values:
                possible_G_values.append(m.G())

print(possible_G_values)
o = Exact_U(Zomega(0,1,0,0)**6, Zomega(0,0,0,0), 4)
t = F*o*F
Id = Exact_U(Zomega(1,0,0,0),Zomega(0,0,0,0),0)

omega = Zomega(0,1,0,0)
possible_u_values = []
possible_outputs = []

for j in range(10):
    possible_u_values.append(omega**j)
    possible_u_values.append(omega**j * Zomega.fromTau(Ztau(0,1)))


for length in range(1,13):
    for i in range(1<<length):
        s=bin(i)[2:]
        s='0'*(length - len(s))+s

        output = Id
        for char in s:
            if char == '0':
                output = output*o
            else:
                output = output*t

        if output.G() <= 18 and output.u in possible_u_values:
            counted = False
            for elt in possible_outputs:
                if output == elt:
                    counted = True
            
            if not counted:
                possible_outputs.append(output)
                print(output)
                print(s)

print(len(possible_outputs))
'''
        

[2, 13, 17, 3, 7, 18]


In [ ]:
o = Exact_U(Zomega(0,1,0,0)**2, Zomega(0,0,0,0), 8)
t = F*o*F
Id = Exact_U(Zomega(1,0,0,0),Zomega(0,0,0,0),5)


for length in range(1,13):
    for i in range(1<<length):
        s=bin(i)[2:]
        s='0'*(length - len(s))+s

        output = Id
        for char in s:
            if char == '0':
                output = output*o
            else:
                output = output*t
        
        if output.u == Zomega(1,0,0,0):
            print(output)
            print(s)

In [ ]:
def exact_synthesize_weave(U):
    g = U.mu()
    V = U
    Circuit = FTCircuit()
    Remainder = FTCircuit()
    f_parity = True

    while g.value() >= 4: # need to be careful this doesn't break from rounding error? if it does change to Gauss complexity which is always an integer
        
        min = 0
        # print("V = " + str(V))
        for j in range(0,5):
            if (F*(T**(2*j))*V).mu().value() < (F*(T**min)*V).mu().value():
                min = 2*j

        V = F*(T**min)*V
        g = V.mu()
        if min != 0:
            Circuit.join("T", 10 - min)
        Circuit.join("F", 1)
        f_parity = not f_parity
    
    # check if |u| = 1
    if V.u*V.u.star() == Zomega(1,0,0,0):
        # factor out phase so that u = 1
        for k in range(10):
            if omega**k == V.u:
                Remainder.addPhase(omega**k)
                V = Exact_U(omega**(10-k), Zomega(0,0,0,0), 5 - 2*k) * V
                break
        
        # Am left with a matrix of the form U(1, 0, k) which we can match to T^j, for j = k-5
        # if j is even and f_parity = True, V is a weave
        if V.k%10 != 5:
            if f_parity:
                if (V.k-5)%2 == 0: 
                    Circuit.join("T", V.k-5)
                    Circuit.addPhase(omega**k)
            else:
                Remainder.join("T", V.k-5)
            
                
    # otherwise |u| = tau
    # We write V = U[w^j t, w^i, k] and find i,j,k
    elif Ztau.fromOmega(V.u*V.u.star()) == Ztau(1, -1):

        # find v
        for i in range(10):
            if omega**i == V.v:
                break

        # find j
        for j in range(10):
            if (omega**(10-j))*V.u == Zomega(0,0,1,-1):
                break
        
        # find k
        k = V.k

        print((j,i,k))

        # now V = w^j T^(-i-j) F T^(-i-j+k)
        if not f_parity:
            if (i-j)%2 == 0 and (-i-j+k)%2 == 0:
                Circuit.join("T",i-j)
                Circuit.join("F", 1)
                Circuit.join("T",-i-j+k)
                Circuit.addPhase(omega**j)

                return (Circuit, Remainder)
        
        if not f_parity:
            if Circuit.remove_F():
                Remainder.join("F", 1)
                
        Remainder.join("T",i-j)
        Remainder.join("F", 1)
        Remainder.join("T",-i-j+k)
        Remainder.addPhase(omega**j)
            
        
    else:
        print("meep! error: V.u not as expected")
    
    return (Circuit, Remainder)

In [7]:
U = T**4 * F * T**2 * F * T**6 * F * T**8 * F * T**2
print(U)
print(U.value())

(circuit, remainder) = exact_synthesize_weave(U)
print(circuit)
print(remainder)
print(circuit.toWeave())
print(circuit.toWeave().numerical_approximate())
# print(np.matmul(circuit.numerical_approximate(), remainder.numerical_approximate()))

U((-2 + 11ω + -14ω² + 7ω³),(7 + -8ω + 3ω² + 3ω³),7)
[[0.409830056250527 - 0.191757838848871*I
  (-0.309016994374948 - 0.951056516295154*I)*(0.414981046245687 - 0.789340856341883*I)]
 [0.414981046245687 + 0.789340856341883*I
  (-0.409830056250525 - 0.191757838848871*I)*(-0.309016994374948 - 0.951056516295154*I)]]
(0, 8, 0)
(1 + 0ω + 0ω² + 0ω³)(T^2)(F)(T^8)(F)(T^6)(F)(T^2)(F)(T^4)
(1 + 0ω + 0ω² + 0ω³)
exp(0.8πi)11222222111111112222111111
[[ 0.40983006-0.19175784j -0.87894396-0.15075069j]
 [ 0.41498105+0.78934086j -0.05572809+0.44902798j]]
